In [83]:
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
from cvxpy import *

In [84]:
risk_free_rate = pd.read_csv(r'C:\Users\Administrator\Desktop\ORIE 5370\Project\check\hw2_risk_free_rate.csv', index_col=None, header=None)
risk_free_rate = np.array(risk_free_rate)
Price = pd.read_csv(r'C:\Users\Administrator\Desktop\ORIE 5370\Project\check\hw2_Price.csv', index_col=None, header=None)
Price = np.mat(Price)
len(risk_free_rate)
len(Price)

967

In [85]:
#Calculate mu and V
##### PRECONDITION: sample_frequency*number_of_samples < trade_date  
def adapted_stats(Price,trade_date,horizon,sample_frequency,number_of_samples,rate_of_decay):  
    h = horizon # 
    P = Price # the Price matrix, each row refer to a date and each column refers to an asset
    t_d = trade_date # the date you trade, 0,1,2...
    s_f = sample_frequency #num
    n_s = number_of_samples #num
    r_d = rate_of_decay #num

    s_d = t_d-1-np.arange(n_s+1)*s_f #reverse chronological
    s_d = s_d[::-1] #chronological    
    #sample dates, a row vector

    S_P = P[s_d,:] #Sample Prices, a matrix

    S_C_R = np.log(S_P[1:,:]/S_P[:-1,:]) #Sample Compound Returns, a matrix
    #now it ’s time to construct the weight
    w = (1.0-float(r_d))**np.arange(1,n_s+1)
    w = w[::-1]
    w = w/sum(w)
    wt = np.transpose([w])  #weights, a non-negative vector that sums to 1
    print "wt", wt
    mean_c_r = S_C_R.T*wt  #mean vector of compound returns
    Cov_C_R = (S_C_R.T*np.diag(w)*S_C_R)-mean_c_r*np.transpose([mean_c_r])
    #covariance matrix of compound returns
    print "mean_c_r:", mean_c_r
    print "Cov_C_R", Cov_C_R
    adapted_mean_c_r = mean_c_r*(float(h)/float(s_f))
    #adapting mean vector to reflect length of holding period
    
    Adapted_Cov_C_R = Cov_C_R*(float(h)/float(s_f))
    #adapting covariance matrix to length of holding period
    print "adapted_mean_c_r:",adapted_mean_c_r
    print "Adapted_Cov_C_R:",Adapted_Cov_C_R
    muu = np.exp(adapted_mean_c_r + 0.5*np.mat(np.diag(Adapted_Cov_C_R)).T)
    print muu
    mu = muu - 1
    #resulting mean vector of ARITHMETIC returns
    
    V = np.multiply((muu*np.transpose([muu.T])),(np.exp(Adapted_Cov_C_R)-1))
    #resulting covariance matrix of ARITHMETIC returns
    return mu,V

In [86]:
def markowitz(mu0,mu,V,sigma,xx0,xx,trans_cost):
    n = len(mu)
    U = np.linalg.cholesky(V)
    x0 = Variable(1)
    x = Variable(n)
    y = Variable(n)
    total_trans_cost = Variable(1)
    objective = Maximize(mu0*x0+mu.T*x)
    constraints = [norm(U*x)<=sigma,
                   x0+sum_entries(x)+total_trans_cost==1,
                   x==xx+y,
                   trans_cost*sum_entries(abs(y))<=total_trans_cost,
                   x0>=0,
                   max_entries(abs(x-1.0/(n+1)))<=0.05,
                   #0.05<=x,
                   #x<=0.2,
                   #x[1]+x[4]<=x[5]+x[6]+x[8],
                   #sum_smallest(x,4)>=-1.25
                   ]
    prob = Problem(objective, constraints)
    result = prob.solve()
    prob.solve()  # Returns the optimal value.
    #print "status:", prob.status
    #print "optimal value", prob.value
    #print "optimal var", x.value, y.value
    return x0.value,x.value

In [87]:
def rebalance_benchmark(benchmark_x0,benchmark_x,trans_cost):
    n = np.size(benchmark_x)
    #print "number of assets:",n
    #print "sum:",sum(benchmark_x)
    z = Variable(1)
    objective = Maximize(z)
    constraints = [(n+1)*z <= benchmark_x0+sum_entries(benchmark_x)-trans_cost*sum_entries(abs(z-benchmark_x))]
    prob = Problem(objective, constraints)
    result = prob.solve(solver=ECOS)
    prob.solve()
    return z.value

In [88]:
###############  BACKTEST  ###############
n = np.size(Price)/len(Price) # n=number of risky assets
e = np.ones([n,1])

###############
#### PARAMETERS
#### (the following choices of parameters can easily be changed)

horizon = 4 # rebalance daily
start = 400 # the day on which you are first given a portfolio to rebalance
number_rebalances = 100 # the number of times the portfolio will be rebalanced 
number_of_samples = 100 # how many samples are to be used 
                      # in computing return avereages and covariances
sample_frequency = 2 # 1 = daily, 2 = , etc.
r_w_f_o_y_e = 0.4 # "relative weight for one year earlier" 
                 # -- a value .4 means that for the (exponential) weights 
                 # used in computing return averages and covariances, 
                 # the weight assigned to the time period one year ago
                 # should be .4 times the weight assigned 
                 # to the most recent period.     
allowable_risk = 1
    # This is the level of risk relative to the benchmark portfolio,
    #   where risk is measured as standard deviation of portfolio returns.
    # Choosing this value to equal 1 means exactly the same amount of risk is allowed,
    # whereas choosing 2 means twice as much risk is allowed as the benchmark, and so on.
trans_cost = 0.005  # transaction cost
wealth = 10000 # initial wealth measured in dollars, including money invested in assets
               # (one dollar invested in an asset is considered as one dollar of wealth,
               #  even though in liquidating the asset, transaction costs would be paid)   
x0 = 0.3 # proportion of wealth in bank initially
x = (0.7/n)*e # proportions in risky assets initially

# Assume the benchmark portfolio is initally equal-weighted, with 1/(n+1) being the 
# proportion of wealth invested in each asset and in the bank.
#### END OF PARAMETERS
######################
#rate_of_decay = 0
rate_of_decay = 1 - r_w_f_o_y_e**(sample_frequency/52.0)
initial_wealth = wealth
benchmark_wealth = wealth
rebalance_dates = start + horizon*np.arange(number_rebalances)

for i in range(len(rebalance_dates)):
    trade_date = rebalance_dates[i]

    ###### REBALANCE YOUR PORTFOLIO AND PAY TRANSACTION COSTS ######
    # It is more natural to rebalance the benchmark portfolio later #

    mu,V = adapted_stats(Price,trade_date,horizon,sample_frequency,number_of_samples,rate_of_decay)
    #[mu,V] = stats(Price,trade_date,sample_frequency,number_of_samples,rate_of_decay)
    mu0 = (1+0.01*risk_free_rate[trade_date-1])**(float(horizon)/52.0)-1
    
    #benchmark_risk = sqrt(quad_form(e, V))/n # there are n assets in the portfolio
    benchmark_risk = sqrt(quad_form(e,V))/(n+1)  # there are n+1 financial instruments
                                                        # including the bank
    sigma = allowable_risk*benchmark_risk
    
    xx0 = x0
    xx = x
    print "mu", mu.T
    x0,x =  markowitz(mu0,mu,V,sigma,xx0,xx,trans_cost)
    print "x0:", x0
    print "x:", x.T
    wealth = wealth*(x0+sum(x))
        # This is the same thing as updating your wealth by subtracting
        # all transaction costs from the rebalancing.  Indeed, in rebalancing,
        # the proportion of your wealth going to trans costs is 1 - x0 - sum(x).  
        
    total = x0+sum(x)
    x0 = x0/total
    x = x/total
        # Rescaling x0 and x so that the sum is 1 (i.e., proportions of current wealth)
    
    ###### PROCEED TO END OF TIME PERIOD AND ACCOUNT FOR GAINS, LOSSES ######
    
    returns = (Price[trade_date+horizon-1,:]-Price[trade_date-1,:])/Price[trade_date-1,:]
        # vector of actual returns for risky assets (this is a row vector)
    
    multiplier = 1+mu0*x0+returns*x
    wealth = multiplier*wealth
            # by leaving off the semicolon, you can watch how wealth changes as the program runs

    if(wealth<=0):
        break   # stops the program if bankruptcy occurs
                # Not needed for benchmark portfolio (because it is long only)
    
    x0 = (1+mu0)*x0/multiplier
    x = np.multiply(x,np.transpose(1+returns))/multiplier
    # these are the proportions of current wealth invested in assets
    
    # Now its time to rebalance the benchmark portfolio and pay transaction costs

    
    benchmark_x0 = (1+mu0)/(n+1)
    benchmark_x = (1+returns)/(n+1)
    # This gives how the equal-weighted portfolio has changed during the time period.
    # The initial unit of wealth has become  benchmark_x0 + sum(benchmark_x).
    # This new level of wealth needs to be distributed equally among the assets and bank.
    # The optimal amount z to put into each one is determined by the following function,
    # which finds the value z so as to minimize transaction costs
    print i,"returns:", returns
    print "benchmark_x:",benchmark_x
    
    z = rebalance_benchmark(benchmark_x0,benchmark_x,trans_cost) 
    benchmark_wealth = benchmark_wealth*(n+1)*z
    print "z:",z

    # Until the end of the next time period, 
    # think of the benchmark portfolio as having been rebalanced
    # with wealth divided equally, that is, the portion of wealth invested in
    # each asset and the bank is 1/(n+1).

wt [[ 0.00108941]
 [ 0.00112848]
 [ 0.00116896]
 [ 0.00121089]
 [ 0.00125433]
 [ 0.00129932]
 [ 0.00134593]
 [ 0.00139421]
 [ 0.00144422]
 [ 0.00149602]
 [ 0.00154969]
 [ 0.00160527]
 [ 0.00166286]
 [ 0.0017225 ]
 [ 0.00178429]
 [ 0.00184829]
 [ 0.00191459]
 [ 0.00198327]
 [ 0.00205441]
 [ 0.0021281 ]
 [ 0.00220444]
 [ 0.00228351]
 [ 0.00236542]
 [ 0.00245027]
 [ 0.00253816]
 [ 0.00262921]
 [ 0.00272352]
 [ 0.00282121]
 [ 0.00292241]
 [ 0.00302724]
 [ 0.00313582]
 [ 0.00324831]
 [ 0.00336482]
 [ 0.00348552]
 [ 0.00361055]
 [ 0.00374006]
 [ 0.00387422]
 [ 0.00401319]
 [ 0.00415714]
 [ 0.00430626]
 [ 0.00446073]
 [ 0.00462073]
 [ 0.00478648]
 [ 0.00495817]
 [ 0.00513602]
 [ 0.00532025]
 [ 0.00551109]
 [ 0.00570878]
 [ 0.00591355]
 [ 0.00612568]
 [ 0.00634541]
 [ 0.00657302]
 [ 0.00680879]
 [ 0.00705303]
 [ 0.00730602]
 [ 0.00756809]
 [ 0.00783956]
 [ 0.00812077]
 [ 0.00841206]
 [ 0.0087138 ]
 [ 0.00902637]
 [ 0.00935015]
 [ 0.00968554]
 [ 0.01003297]
 [ 0.01039285]
 [ 0.01076565]
 [ 0.01

x0: 8.018815404e-13
x: [[ 0.15        0.15        0.05        0.15        0.15        0.05        0.15
   0.05        0.09992949]]
4 returns: [[-0.07644231 -0.07576181 -0.12595228 -0.01633166 -0.03059273 -0.02737226
  -0.04791771 -0.05499439 -0.09459796]]
benchmark_x: [[ 0.09235577  0.09242382  0.08740477  0.09836683  0.09694073  0.09726277
   0.09520823  0.09450056  0.0905402 ]]
z: 0.0945240332264
wt [[ 0.00108941]
 [ 0.00112848]
 [ 0.00116896]
 [ 0.00121089]
 [ 0.00125433]
 [ 0.00129932]
 [ 0.00134593]
 [ 0.00139421]
 [ 0.00144422]
 [ 0.00149602]
 [ 0.00154969]
 [ 0.00160527]
 [ 0.00166286]
 [ 0.0017225 ]
 [ 0.00178429]
 [ 0.00184829]
 [ 0.00191459]
 [ 0.00198327]
 [ 0.00205441]
 [ 0.0021281 ]
 [ 0.00220444]
 [ 0.00228351]
 [ 0.00236542]
 [ 0.00245027]
 [ 0.00253816]
 [ 0.00262921]
 [ 0.00272352]
 [ 0.00282121]
 [ 0.00292241]
 [ 0.00302724]
 [ 0.00313582]
 [ 0.00324831]
 [ 0.00336482]
 [ 0.00348552]
 [ 0.00361055]
 [ 0.00374006]
 [ 0.00387422]
 [ 0.00401319]
 [ 0.00415714]
 [ 0.00430

8 returns: [[-0.08687616 -0.03768681 -0.13146552 -0.0528536  -0.07730852  0.00705716
   0.00914781 -0.01401345 -0.08207934]]
benchmark_x: [[ 0.09131238  0.09623132  0.08685345  0.09471464  0.09226915  0.10070572
   0.10091478  0.09859865  0.09179207]]
z: 0.0953508877144
wt [[ 0.00108941]
 [ 0.00112848]
 [ 0.00116896]
 [ 0.00121089]
 [ 0.00125433]
 [ 0.00129932]
 [ 0.00134593]
 [ 0.00139421]
 [ 0.00144422]
 [ 0.00149602]
 [ 0.00154969]
 [ 0.00160527]
 [ 0.00166286]
 [ 0.0017225 ]
 [ 0.00178429]
 [ 0.00184829]
 [ 0.00191459]
 [ 0.00198327]
 [ 0.00205441]
 [ 0.0021281 ]
 [ 0.00220444]
 [ 0.00228351]
 [ 0.00236542]
 [ 0.00245027]
 [ 0.00253816]
 [ 0.00262921]
 [ 0.00272352]
 [ 0.00282121]
 [ 0.00292241]
 [ 0.00302724]
 [ 0.00313582]
 [ 0.00324831]
 [ 0.00336482]
 [ 0.00348552]
 [ 0.00361055]
 [ 0.00374006]
 [ 0.00387422]
 [ 0.00401319]
 [ 0.00415714]
 [ 0.00430626]
 [ 0.00446073]
 [ 0.00462073]
 [ 0.00478648]
 [ 0.00495817]
 [ 0.00513602]
 [ 0.00532025]
 [ 0.00551109]
 [ 0.00570878]
 [ 0.0

Adapted_Cov_C_R: [[ 0.00343388  0.00241242  0.00287231  0.00243765  0.00218709  0.0009079
   0.00116024  0.00074608  0.00240825]
 [ 0.00241242  0.00341729  0.00181033  0.00156963  0.00165089  0.00078359
   0.00115458  0.0005678   0.00143851]
 [ 0.00287231  0.00181033  0.00551598  0.00258995  0.00224154  0.00106705
   0.00138053  0.00122262  0.00331473]
 [ 0.00243765  0.00156963  0.00258995  0.00216529  0.00186516  0.00076504
   0.00086727  0.00055907  0.00205828]
 [ 0.00218709  0.00165089  0.00224154  0.00186516  0.00230982  0.0008561
   0.00094614  0.00078776  0.0018433 ]
 [ 0.0009079   0.00078359  0.00106705  0.00076504  0.0008561   0.00061796
   0.00057812  0.00055822  0.00083459]
 [ 0.00116024  0.00115458  0.00138053  0.00086727  0.00094614  0.00057812
   0.00158665  0.00062189  0.00089049]
 [ 0.00074608  0.0005678   0.00122262  0.00055907  0.00078776  0.00055822
   0.00062189  0.00117117  0.00092248]
 [ 0.00240825  0.00143851  0.00331473  0.00205828  0.0018433   0.00083459
   0.00

Adapted_Cov_C_R: [[ 0.00300103  0.00222046  0.00254462  0.00202558  0.00187429  0.00069762
   0.00112406  0.00063247  0.00207266]
 [ 0.00222046  0.00318288  0.00177705  0.00139942  0.00156     0.00058669
   0.0011736   0.00047604  0.00136623]
 [ 0.00254462  0.00177705  0.00532301  0.00249183  0.00211752  0.00094918
   0.00129071  0.00111762  0.00307743]
 [ 0.00202558  0.00139942  0.00249183  0.00199179  0.00170306  0.00067436
   0.00074492  0.00050422  0.00186184]
 [ 0.00187429  0.00156     0.00211752  0.00170306  0.00216407  0.00073037
   0.00084628  0.00070308  0.00170588]
 [ 0.00069762  0.00058669  0.00094918  0.00067436  0.00073037  0.00056578
   0.000463    0.00049203  0.00071609]
 [ 0.00112406  0.0011736   0.00129071  0.00074492  0.00084628  0.000463
   0.00146692  0.00053302  0.00082296]
 [ 0.00063247  0.00047604  0.00111762  0.00050422  0.00070308  0.00049203
   0.00053302  0.00099509  0.00081444]
 [ 0.00207266  0.00136623  0.00307743  0.00186184  0.00170588  0.00071609
   0.00

...

adapted_mean_c_r: [[ 0.00753335]
 [ 0.00967641]
 [ 0.01350358]
 [ 0.0124439 ]
 [ 0.0080521 ]
 [ 0.01158107]
 [ 0.00690515]
 [ 0.01439673]
 [ 0.01662396]]
Adapted_Cov_C_R: [[ 0.00256832  0.00224159  0.00214214  0.00188759  0.00157967  0.00077868
   0.00064952  0.00113509  0.0017589 ]
 [ 0.00224159  0.0028877   0.00205784  0.00184425  0.00154428  0.00094399
   0.00082925  0.00114219  0.00165855]
 [ 0.00214214  0.00205784  0.00260169  0.00186854  0.00158171  0.00086545
   0.00078054  0.00119905  0.00181611]
 [ 0.00188759  0.00184425  0.00186854  0.00180493  0.00139867  0.0008235
   0.0007341   0.00107354  0.00157777]
 [ 0.00157967  0.00154428  0.00158171  0.00139867  0.00174728  0.00076959
   0.00059538  0.00092613  0.00147745]
 [ 0.00077868  0.00094399  0.00086545  0.0008235   0.00076959  0.00081882
   0.00062081  0.00076316  0.00083108]
 [ 0.00064952  0.00082925  0.00078054  0.0007341   0.00059538  0.00062081
   0.00092639  0.00067535  0.00073669]
 [ 0.00113509  0.00114219  0.00119905  

z: 0.104896896786
wt [[ 0.00108941]
 [ 0.00112848]
 [ 0.00116896]
 [ 0.00121089]
 [ 0.00125433]
 [ 0.00129932]
 [ 0.00134593]
 [ 0.00139421]
 [ 0.00144422]
 [ 0.00149602]
 [ 0.00154969]
 [ 0.00160527]
 [ 0.00166286]
 [ 0.0017225 ]
 [ 0.00178429]
 [ 0.00184829]
 [ 0.00191459]
 [ 0.00198327]
 [ 0.00205441]
 [ 0.0021281 ]
 [ 0.00220444]
 [ 0.00228351]
 [ 0.00236542]
 [ 0.00245027]
 [ 0.00253816]
 [ 0.00262921]
 [ 0.00272352]
 [ 0.00282121]
 [ 0.00292241]
 [ 0.00302724]
 [ 0.00313582]
 [ 0.00324831]
 [ 0.00336482]
 [ 0.00348552]
 [ 0.00361055]
 [ 0.00374006]
 [ 0.00387422]
 [ 0.00401319]
 [ 0.00415714]
 [ 0.00430626]
 [ 0.00446073]
 [ 0.00462073]
 [ 0.00478648]
 [ 0.00495817]
 [ 0.00513602]
 [ 0.00532025]
 [ 0.00551109]
 [ 0.00570878]
 [ 0.00591355]
 [ 0.00612568]
 [ 0.00634541]
 [ 0.00657302]
 [ 0.00680879]
 [ 0.00705303]
 [ 0.00730602]
 [ 0.00756809]
 [ 0.00783956]
 [ 0.00812077]
 [ 0.00841206]
 [ 0.0087138 ]
 [ 0.00902637]
 [ 0.00935015]
 [ 0.00968554]
 [ 0.01003297]
 [ 0.01039285]
 [ 0

x0: 3.35811295804e-13
x: [[ 0.09896992  0.05        0.15        0.15        0.05        0.15        0.05
   0.15        0.15      ]]
87 returns: [[-0.02092143 -0.02886151  0.         -0.00479904 -0.00291545 -0.02446342
  -0.04724409 -0.00938881 -0.00723634]]
benchmark_x: [[ 0.09790786  0.09711385  0.1         0.0995201   0.09970845  0.09755366
   0.09527559  0.09906112  0.09927637]]
z: 0.0985367250459
wt [[ 0.00108941]
 [ 0.00112848]
 [ 0.00116896]
 [ 0.00121089]
 [ 0.00125433]
 [ 0.00129932]
 [ 0.00134593]
 [ 0.00139421]
 [ 0.00144422]
 [ 0.00149602]
 [ 0.00154969]
 [ 0.00160527]
 [ 0.00166286]
 [ 0.0017225 ]
 [ 0.00178429]
 [ 0.00184829]
 [ 0.00191459]
 [ 0.00198327]
 [ 0.00205441]
 [ 0.0021281 ]
 [ 0.00220444]
 [ 0.00228351]
 [ 0.00236542]
 [ 0.00245027]
 [ 0.00253816]
 [ 0.00262921]
 [ 0.00272352]
 [ 0.00282121]
 [ 0.00292241]
 [ 0.00302724]
 [ 0.00313582]
 [ 0.00324831]
 [ 0.00336482]
 [ 0.00348552]
 [ 0.00361055]
 [ 0.00374006]
 [ 0.00387422]
 [ 0.00401319]
 [ 0.00415714]
 [ 0.00

Adapted_Cov_C_R: [[ 0.00166022  0.00137761  0.00131001  0.0013103   0.00104469  0.00068191
   0.00045875  0.00079424  0.00118636]
 [ 0.00137761  0.00171167  0.00118252  0.00120248  0.00095899  0.00070128
   0.0005223   0.00070859  0.00099205]
 [ 0.00131001  0.00118252  0.00160464  0.00121206  0.00097238  0.00070834
   0.00039383  0.00082075  0.00118628]
 [ 0.0013103   0.00120248  0.00121206  0.00133465  0.00093616  0.00073204
   0.00044     0.00079086  0.00114366]
 [ 0.00104469  0.00095899  0.00097238  0.00093616  0.00108323  0.00057619
   0.00033248  0.00061156  0.00092435]
 [ 0.00068191  0.00070128  0.00070834  0.00073204  0.00057619  0.00089852
   0.00063014  0.00067238  0.00074846]
 [ 0.00045875  0.0005223   0.00039383  0.00044     0.00033248  0.00063014
   0.00101623  0.00057899  0.00044412]
 [ 0.00079424  0.00070859  0.00082075  0.00079086  0.00061156  0.00067238
   0.00057899  0.00090416  0.00083493]
 [ 0.00118636  0.00099205  0.00118628  0.00114366  0.00092435  0.00074846
   0.

z: 0.100462712138
wt [[ 0.00108941]
 [ 0.00112848]
 [ 0.00116896]
 [ 0.00121089]
 [ 0.00125433]
 [ 0.00129932]
 [ 0.00134593]
 [ 0.00139421]
 [ 0.00144422]
 [ 0.00149602]
 [ 0.00154969]
 [ 0.00160527]
 [ 0.00166286]
 [ 0.0017225 ]
 [ 0.00178429]
 [ 0.00184829]
 [ 0.00191459]
 [ 0.00198327]
 [ 0.00205441]
 [ 0.0021281 ]
 [ 0.00220444]
 [ 0.00228351]
 [ 0.00236542]
 [ 0.00245027]
 [ 0.00253816]
 [ 0.00262921]
 [ 0.00272352]
 [ 0.00282121]
 [ 0.00292241]
 [ 0.00302724]
 [ 0.00313582]
 [ 0.00324831]
 [ 0.00336482]
 [ 0.00348552]
 [ 0.00361055]
 [ 0.00374006]
 [ 0.00387422]
 [ 0.00401319]
 [ 0.00415714]
 [ 0.00430626]
 [ 0.00446073]
 [ 0.00462073]
 [ 0.00478648]
 [ 0.00495817]
 [ 0.00513602]
 [ 0.00532025]
 [ 0.00551109]
 [ 0.00570878]
 [ 0.00591355]
 [ 0.00612568]
 [ 0.00634541]
 [ 0.00657302]
 [ 0.00680879]
 [ 0.00705303]
 [ 0.00730602]
 [ 0.00756809]
 [ 0.00783956]
 [ 0.00812077]
 [ 0.00841206]
 [ 0.0087138 ]
 [ 0.00902637]
 [ 0.00935015]
 [ 0.00968554]
 [ 0.01003297]
 [ 0.01039285]
 [ 0

In [89]:
print "your final bank account:", x0 
print "your final risky portfolio:", x 
print "your final wealth:", wealth 
print "benchmark final wealth:", benchmark_wealth 
print "your annualized rate of return:", float(wealth/initial_wealth)**(52.0/(horizon*number_rebalances))-1
benchmark_annual_return = float(benchmark_wealth/initial_wealth)**(52.0/(horizon*number_rebalances))-1
print "benchmark annualized rate of return:", benchmark_annual_return

your final bank account: [[  6.29905387e-13]]
your final risky portfolio: [[ 0.04888108]
 [ 0.04819549]
 [ 0.14950835]
 [ 0.15171289]
 [ 0.15328802]
 [ 0.05165108]
 [ 0.09518347]
 [ 0.15248576]
 [ 0.14909385]]
your final wealth: [[ 15496.69486604]]
benchmark final wealth: 15030.8487805
your annualized rate of return: 0.0585980279835
benchmark annualized rate of return: 0.0544059688082


matrix([[ 1.]])